<a href="https://colab.research.google.com/github/PrzemyslawSarnacki/AirQualityPrediction/blob/master/update_data.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> 

In [1]:
!pip install py-openaq

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import openaq
import warnings
from datetime import timedelta

warnings.simplefilter('ignore')

%matplotlib inline

# Set major seaborn asthetics
sns.set("notebook", style='ticks', font_scale=1.0)

# Increase the quality of inline plots
mpl.rcParams['figure.dpi']= 500


In [3]:
api = openaq.OpenAQ()
resp = api.cities(df=True, limit=10000)

# display the first 10 rows
resp.query("country == 'PL'")

,country,name,city,count,locations
151,PL,Augustów,Augustów,161132871.0,2
223,PL,Belsk,Belsk,21770.0,1
224,PL,Belsk Duży,Belsk Duży,125822709.0,2
248,PL,Biała,Biała,94279227.0,1
249,PL,Biała Podlaska,Biała Podlaska,128518539.0,1
...,...,...,...,...,...
2449,PL,Zielonka,Zielonka,158649172.0,1
2454,PL,Złoty Potok,Złoty Potok,184236679.0,1
2456,PL,Żory,Żory,63993420.0,1
2458,PL,Żyrardów,Żyrardów,64404821.0,2


In [4]:
CITIES = ["Warszawa", "Kraków", "Poznań", "Katowice", "Białystok"]
PARAMETERS = ['pm25', 'pm10', 'no2', 'so2', 'o3', 'co']
LOCATIONS = ["Warszawa-Śródmieście", "Kraków-", "Poznań", "Katowice", "Białystok-Miejska", "WIOŚ Elbląg ul. Bażyńskiego"]

In [5]:
import unicodedata

def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text.lower().replace("ł","l")) if unicodedata.category(c) != 'Mn')


In [11]:
data = {}
indices = [""] + [f".{i}" for i in range(1, 6)]

df = pd.read_csv("https://raw.githubusercontent.com/PrzemyslawSarnacki/AirQualityPrediction/master/data/airq_data_2.csv", index_col=0, parse_dates=True)

for city in CITIES:
    data[strip_accents(city)] = pd.DataFrame(df, columns=[f"{strip_accents(city)}{item}" for item in indices])
    data[strip_accents(city)] = data[strip_accents(city)].rename(columns=data[strip_accents(city)].iloc[0]).drop(data[strip_accents(city)].index[0])
    data[strip_accents(city)] = data[strip_accents(city)].astype(float).interpolate(method="linear")
    data[strip_accents(city)].index = pd.to_datetime(data[strip_accents(city)].index)

for city in CITIES:
  helper_df = pd.DataFrame()
  for param in PARAMETERS:
    last_date = data[strip_accents(city)].index[-1] 
    last_date_str = last_date.strftime("%Y-%m-%d")
    res = api.measurements(city=city, parameter=param, date_from=last_date_str, limit=100000, df=True)
    helper_df[param] = res.resample('D').mean()["value"]
  data[strip_accents(city)] = data[strip_accents(city)].append(helper_df[1:])

output = pd.concat([data[strip_accents(city)] for city in CITIES], axis=1, keys=[strip_accents(city) for city in CITIES])
output.to_csv("airq_data_2.csv")

In [12]:
output

warszawa                        ... bialystok                       
                 pm25       pm10        no2  ...       so2         o3          co
2018-11-21  22.680000  26.282667  51.060667  ...  0.900000  41.200000  296.800000
2018-11-22  25.774699  31.172410  39.961690  ...  0.770833  27.695652  320.750000
2018-11-23  32.482812  38.594687  27.500694  ...  0.670833  25.083333  313.291667
2018-11-24  44.751744  52.565233  22.325455  ...  0.895833  15.541667  441.541667
2018-11-25  57.130000  67.660972  22.806481  ...  0.895833   2.041667  552.250000
...               ...        ...        ...  ...       ...        ...         ...
2021-02-18  38.464364  47.061727  34.776932  ...  1.368182  59.454545  371.500000
2021-02-19  41.226271  49.278305  29.109778  ...  3.958333  59.500000  333.666667
2021-02-20  42.200000  51.962000  47.722500  ...  3.800000  55.000000  495.000000
2021-02-21  38.821727  50.789545  37.345294  ...  1.686364  50.363636  565.772727
2021-02-22  45.746600  55.320400  41.359250  ...  2.530769  37.615385  597.153846

[825 rows x 30 columns]